### Digital Signature Algorithm (DSA)

In [2]:
from random import randint

q = 11
p = 23
g = 4

assert((p - 1) % q == 0)
assert(g >= 2)
assert(g <= (p -2))
assert(pow(g, (p-1)/q) != 1)

print(f"public information looks good. p = {p}, q= {q}, g={g}")

public information looks good. p = 23, q= 11, g=4


In [8]:
alice_private_key = randint(2, q-1)
assert(alice_private_key>=2 & alice_private_key <= (q-1) )
print(f"Alice private key is: {alice_private_key}")

Alice private key is: 4


In [9]:
alice_public_key = pow(g, alice_private_key, p)
print(f"alice public key = {alice_public_key}")

alice public key = 3


now, alice needs to sign this message

In [10]:
hash_dict = {}
def mock_hash_function(inp):
    print(inp)
    if not inp in hash_dict:
        hash_dict[inp] = randint(1, q)
    return hash_dict[inp]

alice_message = "Inspection tommorrow!"
alice_hash = mock_hash_function(alice_message)
print("alice's hash is: ", alice_hash)

Inspection tommorrow!
alice's hash is:  11


Now Alice needs a random per-message secret number k as well as its multiplicative inverse modulo q to compute the signature. There are 2 ways we can compute this modulo inverse:

In [17]:
def modular_inverse(k, q):
    for i in range(0, q):
        if(k * i) % q == 1:
            return i
    print(f"Error! no inverse found")
    return 0

n1=modular_inverse(3,7)
n2=modular_inverse(4,11)
n3=modular_inverse(7,5)
m1=pow(3,-1,7)
m2=pow(4,-1,11)
m3=pow(7,-1,5)

assert(n1 == m1)
assert(n2 == m2)
assert(n3 == m3)
print (f"modular_inverse(3,7) = {m1}")
print (f"modular_inverse(4,11) = {m2}")
print (f"modular_inverse(7,5) = {m3}")

n4 = modular_inverse(2,6)
try:
    m4 = pow(2,-1,6)
except:
    print("exception from pow- no modulo inverse found ..")





modular_inverse(3,7) = 5
modular_inverse(4,11) = 3
modular_inverse(7,5) = 3
Error! no inverse found
exception from pow- no modulo inverse found ..


Now, we can compute the signature:

In [23]:
signed = False
while(signed == False):
    k = randint(1, q-1)
    print("using random k =", k)

    r = pow(g,k,p) % q

    if(r ==0):
        print(f"{k} is not a good value to calculate r, try again..")
        continue

    s = (pow(k,-1, q) * (alice_hash + alice_private_key * r)) % q

    if(s==0):
        print(f"{k} is not a good vlaue to calculate s, try again..")
        continue

    signed = True

signature = (r,s)
print(f"Alice's signature is: {(r,s)}")

using random k = 3
Alice's signature is: (7, 2)


Bob will now re-generate the message, and verify the signature

In [24]:
bob_hash = mock_hash_function(alice_message)

w = (pow(s,-1,q)) % q
u1 = (bob_hash * w) % q
u2 = (r * w) % q
v = ((g**u1 * alice_public_key**u2) %p) %q

if(v == r):
    print("signature valid")
else:
    print("signature invalid")

Inspection tommorrow!
signature valid
